# Not sure if this file is up to date - for operational usage go to Experiment_Automation.ipynb !!

# Test Environment

In [45]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd
#import Models.models as models
#import Models.LSTM.models_LSTM as models
import script_utils as mL
#from Experiment_Class import Experiment
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
import os

NDBC = mL.NDBC_lib
ERA5 = mL.ERA5_lib
Models = mL.Models
DP = mL.DataProcessor
Experiment = mL.Experiment

# %load_ext jupyternotify

# Get Data

In [46]:
data_directory = os.path.join(os.getcwd(), f'data/datasets/type_A')

def build_UI():

    # Select Model-------------------------------------------------------------------------------
    global datafile_widget
    datafile_list = os.listdir(data_directory)
    datafile_widget = widgets.Select(
        options=datafile_list,
        value=datafile_list[0],
        # rows=10,
        description='Datafile:',
        disabled=False
    )

    #STATIONARY_SHIFT -----------------------------------------------------------------------
    global stationaryShift_widget
    stationaryShift_widget = widgets.BoundedIntText(
        value=1,
        min=0,
        max=10,
        step=1,
        description='',
        disabled=False,
    )

    #Supervised:
    n_in_dropdown = widgets.Dropdown(
        options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        value=3,
        description='n_in:',
        disabled=False,
    )
    n_out_dropdown = widgets.Dropdown(
        options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        value=1,
        description='n_out:',
        disabled=False,
    )

    # Test Hours-------------------------------------------------------------------------------
    global test_hours_widget
    test_hours_widget = widgets.IntSlider(
        value=24,
        min=0,
        max=1000,
        step=1,
        description='Test Hours:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )

    # Select Model-------------------------------------------------------------------------------
    global models_widget
    models_list = list(Models.model_dictionary.keys())
    models_widget = widgets.Select(
        options=models_list,
        value=models_list[0],
        rows=10,
        description='Model:',
        disabled=False
    )

    #ALPHA:-------------------------------------------------------------------------------
    # create a FloatSlider widget for a value between 0 and 1
    global alpha_slider
    alpha_slider = widgets.FloatSlider(
        value=0.5,
        min=0,
        max=1,
        step=0.01,
        description='',
        readout_format='.2f',
        orientation='horizontal',
        layout={'width': '500px'}
    )

    left_box = widgets.VBox([
        datafile_widget,
        n_in_dropdown,
        n_out_dropdown,
        widgets.HTML(value="<p>Station Shift:</p>"),
        stationaryShift_widget,
        test_hours_widget
        ],
    )


    right_box = widgets.VBox([
        models_widget
    ])

    display(widgets.HBox([
        left_box,
        right_box,
    ]))
    print("Alpha: ")
    display(alpha_slider)

build_UI() #can be activated whenever needed!

Alpha: 


FloatSlider(value=0.5, layout=Layout(width='500px'), max=1.0, step=0.01)

In [47]:
# DATAFILE = datafile_widget.value
# STATIONARY_SHIFT = stationaryShift_widget.value
# N_TEST_HOURS = test_hours_widget.value
# MODEL_NAME = models_widget.value
# ALPHA = alpha_slider.value

## Optional: use hardcoded variables instead

In [97]:
DATAFILE = "42003_stage_0.pickle"
N_IN=3
N_OUT=1
STATIONARY_SHIFT = 1
N_TEST_HOURS = 31*24 #(One month)
MODEL_NAME = "LSTM_2"
ALPHA = 1

In [98]:
print(f"Datafile: {DATAFILE}")
print(f"Alpha: {ALPHA}")
print(f"Stationary Shift: {STATIONARY_SHIFT}")
print(f"N_IN: {N_IN}")
print(f"N_OUT: {N_OUT}")
print(f"Test-Hours: {N_TEST_HOURS}")
print(f"Model: {MODEL_NAME}")

Datafile: 42003_stage_0.pickle
Alpha: 1
Stationary Shift: 1
N_IN: 3
N_OUT: 1
Test-Hours: 744
Model: LSTM_2


In [99]:
with open(f'data/datasets/type_A/{DATAFILE}', 'rb') as f:
    # load the object from the file using pickle.load()
    dataset = pickle.load(f)

print("stations",dataset["stations"])
print("years",dataset["years"])
print("nan_threshold",dataset["nan_threshold"])
print("features",dataset["features"])
print("add_era5",dataset["add_era5"])

data = dataset["data"]
data

stations ['42003']
years ['2017', '2018', '2019', '2020', '2021', '2022']
nan_threshold 0.66
features ['WDIR', 'WSPD', 'WVHT', 'APD', 'MWD', 'PRES', 'ATMP', 'WTMP', 'DEWP']
add_era5 True


,WDIR_42003,WSPD_42003,PRES_42003,ATMP_42003,WTMP_42003,DEWP_42003,PRES_42003_ERA5,DEWP_42003_ERA5,ATMP_42003_ERA5,WDIR_42003_ERA5,WTMP_42003_ERA5,WSPD_42003_ERA5
2017-01-01 00:00:00,52.0,1.2,1017.0,23.9,24.8,24.8,1017.543815,18.389685,25.496719,121.924126,25.576055,8.435547
2017-01-01 01:00:00,52.0,1.2,1017.0,23.9,24.8,24.8,1017.844245,18.258847,25.561329,121.222618,25.576055,8.621167
2017-01-01 02:00:00,52.0,1.2,1017.0,23.9,24.8,24.8,1018.266168,18.152943,25.712758,119.223252,25.576055,8.626273
2017-01-01 03:00:00,52.0,1.2,1017.0,23.9,24.8,24.8,1018.731300,18.085380,25.847811,118.280718,25.576055,8.543633
2017-01-01 04:00:00,52.0,1.2,1017.0,23.9,24.8,24.8,1018.775526,18.327483,25.777593,116.010992,25.576055,8.382881
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,54.0,3.4,1012.1,28.7,29.4,25.1,1017.354549,22.430594,25.116762,62.437416,26.043836,5.699986
2022-12-31 20:00:00,54.0,3.4,1012.1,28.7,29.4,25.1,1017.170520,22.323688,24.950898,52.297944,26.043836,5.119160
2022-12-31 21:00:00,54.0,3.4,1012.1,28.7,29.4,25.1,1017.046382,21.885496,23.590570,38.390185,26.043836,4.777839
2022-12-31 22:00:00,54.0,3.4,1012.1,28.7,29.4,25.1,1018.447468,21.882078,23.937287,343.208740,26.256956,3.422273


# Data Processing

## 1. Make data stationary

In [100]:
STATIONARY = True #Set Flag for report
data_stationary = DP.data_to_stationary(data, n = STATIONARY_SHIFT)
data_stationary.head()

,WDIR_42003,WSPD_42003,PRES_42003,ATMP_42003,WTMP_42003,DEWP_42003,PRES_42003_ERA5,DEWP_42003_ERA5,ATMP_42003_ERA5,WDIR_42003_ERA5,WTMP_42003_ERA5,WSPD_42003_ERA5
2017-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.300430,-0.130838,0.064610,-0.701509,0.0,0.185620
2017-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.421923,-0.105904,0.151429,-1.999366,0.0,0.005106
2017-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.465132,-0.067564,0.135053,-0.942534,0.0,-0.082641
2017-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.044226,0.242104,-0.070218,-2.269726,0.0,-0.160752
2017-01-01 05:00:00,0.0,0.0,0.0,0.0,0.0,0.0,-0.210453,0.067028,-0.067302,-1.705968,0.0,-0.302338


## 2. Transform to supervised problem

In [101]:
data_supervised = DP.data_to_supervised(data_stationary, n_in=N_IN, n_out=N_OUT)
data_supervised.head()

,WDIR_42003(t-3),WSPD_42003(t-3),PRES_42003(t-3),ATMP_42003(t-3),WTMP_42003(t-3),DEWP_42003(t-3),PRES_42003_ERA5(t-3),DEWP_42003_ERA5(t-3),ATMP_42003_ERA5(t-3),WDIR_42003_ERA5(t-3),...,PRES_42003(t),ATMP_42003(t),WTMP_42003(t),DEWP_42003(t),PRES_42003_ERA5(t),DEWP_42003_ERA5(t),ATMP_42003_ERA5(t),WDIR_42003_ERA5(t),WTMP_42003_ERA5(t),WSPD_42003_ERA5(t)
2017-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.300430,-0.130838,0.064610,-0.701509,...,0.0,0.0,0.0,0.0,0.044226,0.242104,-0.070218,-2.269726,0.0,-0.160752
2017-01-01 05:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.421923,-0.105904,0.151429,-1.999366,...,0.0,0.0,0.0,0.0,-0.210453,0.067028,-0.067302,-1.705968,0.0,-0.302338
2017-01-01 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.465132,-0.067564,0.135053,-0.942534,...,0.0,0.0,0.0,0.0,-0.497666,0.426295,-0.085473,0.483651,0.0,-0.071808
2017-01-01 07:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.044226,0.242104,-0.070218,-2.269726,...,0.0,0.0,0.0,0.0,-0.375664,0.118773,-0.019518,-1.945436,0.0,-0.151994
2017-01-01 08:00:00,0.0,0.0,0.0,0.0,0.0,0.0,-0.210453,0.067028,-0.067302,-1.705968,...,0.0,0.0,0.0,0.0,-0.150469,0.194380,-0.021985,0.439287,0.0,-0.172049


## train test split

In [102]:
train_X, train_y, test_X, test_y = DP.train_test_split(data_supervised, N_TEST_HOURS)
print("Shapes: ", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

Shapes:  (51836, 3, 12) (51836, 12) (744, 3, 12) (744, 12)


In [96]:
type(train_y)

numpy.ndarray

In [91]:
#Developement experiment:
# Flatten X to fix issue with supervised problem when n_out > 1
#train_X = train_X.reshape((train_X.shape[0], -1))
#test_X = test_X.reshape((test_X.shape[0], -1))

#print("Shapes: ", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [90]:
train_X.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

## Normalize the data

In [87]:
# Scale the data
#NORMALIZED = True   #set flag for report

#train_X_scaled, train_y_scaled, test_X_scaled, test_y_scaled, SCALER = DP.scale_data(train_X,
#                                                                                     train_y,
#                                                                                     test_X,
#                                                                                     test_y)

train_X_scaled, train_y_scaled, test_X_scaled, test_y_scaled = train_X, train_y, test_X, test_y #just to exclude scaling for now!




---
# Select Model

In [88]:
print(train_X_scaled.shape)
print(train_y_scaled.shape)

(51792, 288)
(51792, 288)


In [89]:
model = Models.get_model(MODEL_NAME, train_X_scaled, train_y_scaled, ALPHA)

IndexError: tuple index out of range

----
# One-Shot Forecasting

In [35]:
_ = model.predict(train_X_scaled, batch_size=1)

NameError: name 'model' is not defined

In [16]:
output_cols = data.columns.tolist()
output_cols

['WDIR_42001',
 'WSPD_42001',
 'PRES_42001',
 'ATMP_42001',
 'WTMP_42001',
 'DEWP_42001',
 'WDIR_42002',
 'WSPD_42002',
 'PRES_42002',
 'ATMP_42002',
 'WTMP_42002',
 'DEWP_42002',
 'WDIR_42003',
 'WSPD_42003',
 'PRES_42003',
 'ATMP_42003',
 'WTMP_42003',
 'DEWP_42003',
 'WDIR_42019',
 'WSPD_42019',
 'PRES_42019',
 'ATMP_42019',
 'WTMP_42019',
 'DEWP_42019',
 'WDIR_42020',
 'WSPD_42020',
 'PRES_42020',
 'ATMP_42020',
 'WTMP_42020',
 'DEWP_42020',
 'WDIR_42035',
 'WSPD_42035',
 'PRES_42035',
 'ATMP_42035',
 'WTMP_42035',
 'DEWP_42035',
 'WDIR_42036',
 'WSPD_42036',
 'PRES_42036',
 'ATMP_42036',
 'DEWP_42036',
 'WDIR_42039',
 'WSPD_42039',
 'PRES_42039',
 'ATMP_42039',
 'WTMP_42039',
 'WDIR_42040',
 'WSPD_42040',
 'PRES_42040',
 'ATMP_42040',
 'WTMP_42040',
 'DEWP_42040',
 'WTMP_42001_ERA5',
 'ATMP_42020_ERA5',
 'ATMP_42001_ERA5',
 'DEWP_42001_ERA5',
 'PRES_42019_ERA5',
 'ATMP_42040_ERA5',
 'PRES_42001_ERA5',
 'WDIR_42035_ERA5',
 'PRES_42003_ERA5',
 'DEWP_42003_ERA5',
 'WTMP_42035_ERA5',


In [17]:
yhat = model.predict(test_X_scaled)
# yhat_unscaled = DP.invert_scaling(yhat, SCALER)
yhat_unscaled = yhat # since test_X_scaled is actually not scaled - lol :D
yhat_unscaled_df = pd.DataFrame(yhat_unscaled, columns=[name + "_pred" for name in output_cols])
yhat_unscaled_df.set_index(data.tail(len(yhat)).index, inplace=True)
yhat_unscaled_df

1/1 [==============================] - 0s 200ms/step


,WDIR_42001_pred,WSPD_42001_pred,PRES_42001_pred,ATMP_42001_pred,WTMP_42001_pred,DEWP_42001_pred,WDIR_42002_pred,WSPD_42002_pred,PRES_42002_pred,ATMP_42002_pred,...,WDIR_42039_ERA5_pred,WSPD_42039_ERA5_pred,ATMP_42039_ERA5_pred,WSPD_42035_ERA5_pred,WSPD_42001_ERA5_pred,DEWP_42020_ERA5_pred,ATMP_42019_ERA5_pred,WTMP_42039_ERA5_pred,WSPD_42002_ERA5_pred,PRES_42039_ERA5_pred
2022-12-31 00:00:00,0.083912,-0.048608,0.017255,-0.020981,-0.029096,0.074749,0.016794,0.120610,-0.054225,-0.013221,...,-0.032019,0.000623,-0.009460,0.005582,0.394607,-0.002391,-0.000916,0.005079,0.003192,0.002654
2022-12-31 01:00:00,0.045644,-0.040685,0.014082,-0.030722,-0.027556,0.066822,-0.006201,0.138422,-0.053979,-0.017088,...,-0.015483,-0.001127,-0.013398,0.001753,0.632048,-0.001010,-0.001129,0.001962,0.000533,0.000526
2022-12-31 02:00:00,0.003008,-0.030363,0.014721,-0.048893,-0.024733,0.057439,-0.034303,0.155170,-0.057559,-0.018567,...,0.005179,-0.003076,-0.018241,-0.002980,0.929440,0.000662,-0.001427,-0.002222,-0.002966,-0.002195
2022-12-31 03:00:00,-0.049466,-0.020243,0.007982,-0.063162,-0.018153,0.045537,-0.059552,0.176598,-0.053096,-0.025460,...,0.026563,-0.005395,-0.023194,-0.007914,1.235240,0.002744,-0.001897,-0.006352,-0.006578,-0.004993
2022-12-31 04:00:00,-0.049283,-0.020291,0.007996,-0.063090,-0.018179,0.045587,-0.059456,0.176524,-0.053113,-0.025450,...,0.026479,-0.005386,-0.023175,-0.007895,1.234046,0.002736,-0.001894,-0.006335,-0.006563,-0.004982
2022-12-31 05:00:00,-0.043748,-0.021754,0.008409,-0.060918,-0.018993,0.047126,-0.056524,0.174228,-0.053601,-0.025140,...,0.023929,-0.005128,-0.022595,-0.007318,1.197709,0.002480,-0.001824,-0.005831,-0.006111,-0.004639
2022-12-31 06:00:00,-0.010879,-0.029063,0.010343,-0.049201,-0.021766,0.053588,-0.037850,0.162315,-0.053867,-0.022757,...,0.009470,-0.003690,-0.019228,-0.003998,0.990672,0.001252,-0.001590,-0.002968,-0.003689,-0.002771
2022-12-31 07:00:00,-0.005338,-0.028895,0.013744,-0.051033,-0.023621,0.055427,-0.038318,0.158678,-0.057040,-0.019752,...,0.008588,-0.003446,-0.019032,-0.003769,0.978391,0.000998,-0.001499,-0.002887,-0.003544,-0.002649
2022-12-31 08:00:00,0.016172,-0.035140,0.012925,-0.040408,-0.024201,0.060338,-0.022776,0.149657,-0.054874,-0.019494,...,-0.002391,-0.002434,-0.016451,-0.001262,0.821256,0.000186,-0.001320,-0.000718,-0.001700,-0.001274
2022-12-31 09:00:00,0.043070,-0.040825,0.014812,-0.031543,-0.026777,0.066374,-0.007714,0.138335,-0.055047,-0.016859,...,-0.014261,-0.001222,-0.013670,0.001470,0.650719,-0.000867,-0.001107,0.001617,0.000305,0.000275


In [18]:
# # Get the list of columns starting with "WDIR"
# wdir_columns = [col for col in yhat_unscaled_df.columns if col.startswith("WDIR")]
#
# # Modify the values in the selected columns
# yhat_unscaled_df[wdir_columns] = yhat_unscaled_df[wdir_columns] % 360
#
# yhat_unscaled_df

In [19]:
evaluation_1 = data.tail(len(yhat)+1).copy()  #+1 since i need that value for de-differencing
evaluation_1

,WDIR_42001,WSPD_42001,PRES_42001,ATMP_42001,WTMP_42001,DEWP_42001,WDIR_42002,WSPD_42002,PRES_42002,ATMP_42002,...,WDIR_42039_ERA5,WSPD_42039_ERA5,ATMP_42039_ERA5,WSPD_42035_ERA5,WSPD_42001_ERA5,DEWP_42020_ERA5,ATMP_42019_ERA5,WTMP_42039_ERA5,WSPD_42002_ERA5,PRES_42039_ERA5
2022-12-30 23:00:00,185.0,5.9,1012.9,25.3,24.2,24.4,322.0,5.5,1011.7,24.7,...,113.871622,10.542640,23.935086,2.632245,5.301299,18.452689,19.872416,24.392457,3.651173,1015.146367
2022-12-31 00:00:00,189.0,5.0,1013.5,25.2,24.3,24.5,333.0,5.6,1012.3,24.6,...,108.235067,10.136320,24.123266,3.286975,5.057694,18.248043,20.009884,24.392457,1.420595,1015.755665
2022-12-31 01:00:00,203.0,5.6,1013.8,24.7,24.3,23.5,359.0,4.6,1012.9,24.6,...,97.574364,9.799587,24.028398,4.023334,5.587926,17.794162,20.081088,24.392457,1.134690,1016.328747
2022-12-31 02:00:00,169.0,2.3,1014.5,24.8,24.3,24.4,18.0,4.2,1013.5,24.7,...,84.710673,10.336202,22.973194,3.117344,5.840792,17.447368,20.151057,24.392457,1.613154,1016.790195
2022-12-31 03:00:00,53.0,6.0,1014.8,22.7,24.4,22.1,51.0,4.0,1014.1,24.5,...,70.760477,11.014376,23.074671,1.771335,6.463279,17.274733,20.047338,24.392457,1.853178,1017.056071
2022-12-31 04:00:00,55.0,8.5,1015.2,22.3,24.4,21.9,69.0,3.1,1014.4,24.3,...,61.682710,9.975791,23.811448,1.624550,6.500897,17.075041,19.827142,24.392457,2.384701,1017.194548
2022-12-31 05:00:00,72.0,7.6,1014.9,22.6,24.3,22.1,89.0,3.7,1014.4,24.3,...,65.365573,9.259451,24.030731,1.979889,6.541557,16.895165,19.694613,24.392457,3.354966,1016.693048
2022-12-31 06:00:00,77.0,7.2,1014.4,23.0,24.4,23.0,118.0,2.6,1014.5,24.4,...,77.495898,8.763196,24.011680,2.840376,6.465415,16.681753,19.667860,24.392457,4.305897,1016.111445
2022-12-31 07:00:00,97.0,7.8,1014.1,23.1,24.4,23.0,151.0,2.8,1014.6,24.5,...,81.234776,9.032894,24.101493,3.889530,6.487022,16.582288,19.685558,24.392457,4.524105,1015.494477
2022-12-31 08:00:00,148.0,3.3,1013.9,24.4,24.4,24.3,195.0,1.2,1014.8,24.6,...,77.326712,9.596466,24.324664,3.860175,6.690521,16.566664,19.631641,24.392457,4.233950,1015.054333


In [20]:
for col in evaluation_1.columns:
    evaluation_1[f"{col}_pred"]= evaluation_1[col].shift(STATIONARY_SHIFT) + yhat_unscaled_df[f"{col}_pred"]

evaluation_1 = evaluation_1.iloc[STATIONARY_SHIFT:]  # remove first n entries since there is no delta value for them
evaluation_1

/var/folders/ys/lj6w647n5qjdv23zdxs_fn740000gn/T/ipykernel_83941/4009267436.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  evaluation_1[f"{col}_pred"]= evaluation_1[col].shift(STATIONARY_SHIFT) + yhat_unscaled_df[f"{col}_pred"]
/var/folders/ys/lj6w647n5qjdv23zdxs_fn740000gn/T/ipykernel_83941/4009267436.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  evaluation_1[f"{col}_pred"]= evaluation_1[col].shift(STATIONARY_SHIFT) + yhat_unscaled_df[f"{col}_pred"]
/var/folders/ys/lj6w647n5qjdv23zdxs_fn740000gn/T/ipykernel_83941/400

,WDIR_42001,WSPD_42001,PRES_42001,ATMP_42001,WTMP_42001,DEWP_42001,WDIR_42002,WSPD_42002,PRES_42002,ATMP_42002,...,WDIR_42039_ERA5_pred,WSPD_42039_ERA5_pred,ATMP_42039_ERA5_pred,WSPD_42035_ERA5_pred,WSPD_42001_ERA5_pred,DEWP_42020_ERA5_pred,ATMP_42019_ERA5_pred,WTMP_42039_ERA5_pred,WSPD_42002_ERA5_pred,PRES_42039_ERA5_pred
2022-12-31 00:00:00,189.0,5.0,1013.5,25.2,24.3,24.5,333.0,5.6,1012.3,24.6,...,113.839604,10.543263,23.925626,2.637827,5.695906,18.450298,19.871500,24.397536,3.654365,1015.149021
2022-12-31 01:00:00,203.0,5.6,1013.8,24.7,24.3,23.5,359.0,4.6,1012.9,24.6,...,108.219584,10.135193,24.109868,3.288728,5.689742,18.247033,20.008756,24.394419,1.421128,1015.756191
2022-12-31 02:00:00,169.0,2.3,1014.5,24.8,24.3,24.4,18.0,4.2,1013.5,24.7,...,97.579543,9.796511,24.010157,4.020354,6.517366,17.794824,20.079661,24.390235,1.131724,1016.326552
2022-12-31 03:00:00,53.0,6.0,1014.8,22.7,24.4,22.1,51.0,4.0,1014.1,24.5,...,84.737236,10.330807,22.950001,3.109430,7.076032,17.450112,20.149160,24.386105,1.606576,1016.785202
2022-12-31 04:00:00,55.0,8.5,1015.2,22.3,24.4,21.9,69.0,3.1,1014.4,24.3,...,70.786955,11.008990,23.051497,1.763439,7.697326,17.277469,20.045444,24.386122,1.846616,1017.051089
2022-12-31 05:00:00,72.0,7.6,1014.9,22.6,24.3,22.1,89.0,3.7,1014.4,24.3,...,61.706639,9.970663,23.788853,1.617232,7.698606,17.077521,19.825319,24.386627,2.378590,1017.189909
2022-12-31 06:00:00,77.0,7.2,1014.4,23.0,24.4,23.0,118.0,2.6,1014.5,24.4,...,65.375044,9.255761,24.011504,1.975891,7.532228,16.896417,19.693023,24.389489,3.351278,1016.690277
2022-12-31 07:00:00,97.0,7.8,1014.1,23.1,24.4,23.0,151.0,2.8,1014.6,24.5,...,77.504486,8.759750,23.992648,2.836607,7.443806,16.682751,19.666361,24.389570,4.302353,1016.108796
2022-12-31 08:00:00,148.0,3.3,1013.9,24.4,24.4,24.3,195.0,1.2,1014.8,24.6,...,81.232386,9.030461,24.085041,3.888268,7.308278,16.582474,19.684238,24.391739,4.522405,1015.493202
2022-12-31 09:00:00,195.0,5.4,1014.0,25.0,24.4,24.6,207.0,2.3,1014.4,24.5,...,77.312451,9.595243,24.310994,3.861645,7.341240,16.565797,19.630534,24.394075,4.234256,1015.054608


# Correct wind direction (modulo 360)

In [21]:
# Get the list of columns starting with "WDIR"
wdir_columns = [col for col in evaluation_1.columns if col.startswith("WDIR")]

# Modify the values in the selected columns
evaluation_1[wdir_columns] = evaluation_1[wdir_columns] % 360

evaluation_1

/var/folders/ys/lj6w647n5qjdv23zdxs_fn740000gn/T/ipykernel_83941/1134168289.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  evaluation_1[wdir_columns] = evaluation_1[wdir_columns] % 360


,WDIR_42001,WSPD_42001,PRES_42001,ATMP_42001,WTMP_42001,DEWP_42001,WDIR_42002,WSPD_42002,PRES_42002,ATMP_42002,...,WDIR_42039_ERA5_pred,WSPD_42039_ERA5_pred,ATMP_42039_ERA5_pred,WSPD_42035_ERA5_pred,WSPD_42001_ERA5_pred,DEWP_42020_ERA5_pred,ATMP_42019_ERA5_pred,WTMP_42039_ERA5_pred,WSPD_42002_ERA5_pred,PRES_42039_ERA5_pred
2022-12-31 00:00:00,189.0,5.0,1013.5,25.2,24.3,24.5,333.0,5.6,1012.3,24.6,...,113.839604,10.543263,23.925626,2.637827,5.695906,18.450298,19.871500,24.397536,3.654365,1015.149021
2022-12-31 01:00:00,203.0,5.6,1013.8,24.7,24.3,23.5,359.0,4.6,1012.9,24.6,...,108.219584,10.135193,24.109868,3.288728,5.689742,18.247033,20.008756,24.394419,1.421128,1015.756191
2022-12-31 02:00:00,169.0,2.3,1014.5,24.8,24.3,24.4,18.0,4.2,1013.5,24.7,...,97.579543,9.796511,24.010157,4.020354,6.517366,17.794824,20.079661,24.390235,1.131724,1016.326552
2022-12-31 03:00:00,53.0,6.0,1014.8,22.7,24.4,22.1,51.0,4.0,1014.1,24.5,...,84.737236,10.330807,22.950001,3.109430,7.076032,17.450112,20.149160,24.386105,1.606576,1016.785202
2022-12-31 04:00:00,55.0,8.5,1015.2,22.3,24.4,21.9,69.0,3.1,1014.4,24.3,...,70.786955,11.008990,23.051497,1.763439,7.697326,17.277469,20.045444,24.386122,1.846616,1017.051089
2022-12-31 05:00:00,72.0,7.6,1014.9,22.6,24.3,22.1,89.0,3.7,1014.4,24.3,...,61.706639,9.970663,23.788853,1.617232,7.698606,17.077521,19.825319,24.386627,2.378590,1017.189909
2022-12-31 06:00:00,77.0,7.2,1014.4,23.0,24.4,23.0,118.0,2.6,1014.5,24.4,...,65.375044,9.255761,24.011504,1.975891,7.532228,16.896417,19.693023,24.389489,3.351278,1016.690277
2022-12-31 07:00:00,97.0,7.8,1014.1,23.1,24.4,23.0,151.0,2.8,1014.6,24.5,...,77.504486,8.759750,23.992648,2.836607,7.443806,16.682751,19.666361,24.389570,4.302353,1016.108796
2022-12-31 08:00:00,148.0,3.3,1013.9,24.4,24.4,24.3,195.0,1.2,1014.8,24.6,...,81.232386,9.030461,24.085041,3.888268,7.308278,16.582474,19.684238,24.391739,4.522405,1015.493202
2022-12-31 09:00:00,195.0,5.4,1014.0,25.0,24.4,24.6,207.0,2.3,1014.4,24.5,...,77.312451,9.595243,24.310994,3.861645,7.341240,16.565797,19.630534,24.394075,4.234256,1015.054608


# Recurrent forecast (EXCLUDED FOR NOW!)

In [22]:
# model.reset_states()
# _ = model.predict(train_X_scaled, batch_size=1)

In [23]:
# # make a one-step forecast
# # This function helps with reshaping.
# def single_forecast(model, x):
#     x = x.reshape(1, 1, len(x[0]))
#     yhat = model.predict(x, verbose=0)
#     return yhat

In [24]:
# # Prepare data structure
# prediction_2 = pd.DataFrame(test_y_scaled, columns=output_cols)
# for col in output_cols:
#     prediction_2[f"{col}_pred"] = 0
#
# prediction_2

In [25]:
# #Forecast a whole week
# prev_obs = test_X_scaled[0]
#
# for i, row in prediction_2.iterrows():
#     yhat = single_forecast(model, prev_obs)
#     prev_obs = yhat
#
#     #Mapping of array index and df column name
#     for j, element in enumerate(output_cols):
#         prediction_2.at[i, f"{element}_pred"] =yhat[0,j]
#
# # prediction_2.plot(kind='line')
# prediction_2

In [26]:
# #Reverse differenciate
# first_row = data.iloc[-len(yhat)-1]
#
# yhat = prediction_2[[name + "_pred" for name in output_cols]].values
# yhat_unscaled = DP.invert_scaling(yhat, SCALER)
# yhat_true_value = DP.stationary_to_data(yhat_unscaled, first_row)
#
# yhat_true_value_df = pd.DataFrame(yhat_true_value, columns=[name + "_pred" for name in output_cols])
# yhat_true_value_df.set_index(data.tail(len(yhat)).index, inplace=True)
#
# true_value = data.tail(len(yhat)+1).copy()
#
# evaluation_2 = pd.concat([true_value, yhat_true_value_df], axis=1)
# evaluation_2 = evaluation_2.iloc[STATIONARY_SHIFT:]
# evaluation_2

In [27]:
# wtmp_true = [col for col in evaluation_2.columns if col.startswith("WTMP")][0]
#
# mae_2 = mean_absolute_error(evaluation_2[wtmp_true], evaluation_2[f"{wtmp_true}_pred"])
# mse_2 = mean_squared_error(evaluation_2[wtmp_true], evaluation_2[f"{wtmp_true}_pred"])
# print('MAE: ', mae_2)
# print('MSE: ', mse_2)

In [28]:
#evaluation_2.plot(kind='line')

# SAVE

In [29]:
%%notify -m "Finished!!"
print("reached checkpoint")

reached checkpoint


<IPython.core.display.Javascript object>

In [31]:
# create a text input widget for username
filename_widget = widgets.Text(
    value='',
    placeholder='Enter filename',
    description='Filename:',
    disabled=False
)
# add '.csv' to the description
extension_label = widgets.Label('.pickle')

# display the widget
display(widgets.HBox([filename_widget, extension_label]))

print("Please also check if the reports description needs to be changed!")

Please also check if the reports description needs to be changed!


In [32]:
report_description="FIRST TRANSFORMER EXPERIMENT"

In [34]:
filename = filename_widget.value
if filename == "":
    print("Enter a valid filename!")

else:
    #Save Data About executed Test:

    # Convert model summary to string
    stringlist = []
    model.summary(print_fn=lambda x: stringlist.append(x))
    model_summary = "\n".join(stringlist)

    report = Experiment(
        name=filename,
        description=report_description,

        stations = dataset["stations"],
        years = dataset["years"],
        nan_threshold=dataset["nan_threshold"],
        features=dataset["features"],
        era5=dataset["add_era5"],

        stationary_shift=STATIONARY_SHIFT,

        n_test_hours=N_TEST_HOURS,

        #stationary=STATIONARY,
        scaler= None, # SCALER,

        model_name = MODEL_NAME,
        model_summary=model_summary,

        one_shot_forecast = evaluation_1,
        recursive_forecast = None,   # evaluation_2
        training_time = 0.0
    )


    # open a file for writing in binary mode
    filepath = f'data/reports/publication/{report.name}.pickle'
    with open(filepath, 'wb') as f:
        # write the object to the file using pickle.dump()
        pickle.dump(report, f)
        print("File successfully saved:")
        print(filepath)

File successfully saved:
data/reports/publication/transformer_0.pickle
